# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [47]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [48]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,severo-kurilsk,92,RU,1534988024,93,50.68,156.12,54.27,2.73
1,1,darhan,8,MN,1534988024,71,49.49,105.92,72.81,8.55
2,2,tarakan,48,ID,1534988024,100,3.30,117.63,80.10,4.41
3,3,komsomolskiy,48,RU,1534987754,72,67.55,63.78,54.46,15.46
4,4,souillac,0,FR,1534986000,73,45.60,-0.60,71.60,3.36


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [49]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    framde_width = 500,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5
)

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [50]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df.loc[(city_data_df["Max Temp"] <= 27) & (city_data_df["Max Temp"] >= 21) & (city_data_df["Wind Speed"] <= 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_weather = ideal_weather.dropna()

# Display sample data
ideal_weather

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
12,12,carnarvon,0,ZA,1534988027,66,-30.97,22.13,24.3,2.84
458,458,de aar,0,ZA,1534988184,63,-30.65,24.01,27.0,2.84


In [51]:
len(ideal_weather)

2

### Step 3: Create a new DataFrame called `hotel_df`.

In [52]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
12,carnarvon,ZA,-30.97,22.13,66,
458,de aar,ZA,-30.65,24.01,63,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [55]:
radius = 10000
categories = "accommodation.hotel"

params = {
   "categories":categories,
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df.loc[index, "Lng"]
    latitude = hotel_df.loc[index, "Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
carnarvon - nearest hotel: Carnarvon Hotel
de aar - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
12,carnarvon,ZA,-30.97,22.13,66,Carnarvon Hotel
458,de aar,ZA,-30.65,24.01,63,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [56]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    hover_cols = ["Hotel Name", "Country"]
)
# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)